# Q1

Given the following short documents (1-8), each labeled with a class:
1. banana, carrot, cucumber, pea : vegetable
2. school, pea, rose, lily, basket : 
ower
3. banana, pea, potato, lotus : vegetable
4. hibiscus, grape, potato, mango, apple : fruit
5. hibiscus, lotus, lily, apple, banana : fruit
6. rose, hibiscus, banana, rose : 
ower
7. rose, rose, rose, cucumber : 
ower
8. carrot, mango, grape, rose : fruit

And documents:
1. D1 : rose, lily, apple, carrot
2. D2 : pea, pea, lotus, grape
Compute the most likely class for D1 and D2. Assume a Naive Bayes classier and use
add-lambda smoothing (with lambda = 0.1) for the likelihood.

In [1]:
import re
import math

In [2]:
data = """
1. banana, carrot, cucumber, pea : vegetable
2. school, pea, rose, lily, basket : flower
3. banana, pea, potato, lotus : vegetable
4. hibiscus, grape, potato, mango, apple : fruit
5. hibiscus, lotus, lily, apple, banana : fruit
6. rose, hibiscus, banana, rose : flower
7. rose, rose, rose, cucumber : flower
8. carrot, mango, grape, rose : fruit
"""

In [3]:
doc = """
1. rose, lily, apple, carrot : D1
2. pea, pea, lotus, grape : D2
"""

In [4]:
def function_to_format_data(data):
    dict_label = {}
    data = data.split("\n")
    number_docs = 0
    count_p_label = {}
    for each_line in data:
        if each_line:
            data_in_each_line = each_line.strip().split(". ")[1].split(" : ")
            label = data_in_each_line[1]
            if label not in dict_label:
                dict_label[label] = data_in_each_line[0].split(", ")
                count_p_label[label] = 1
            else:
                dict_label[label].extend(data_in_each_line[0].split(", "))
                count_p_label[label] += 1
            number_docs += 1
    print (count_p_label)
    count_per_label = {i:count_p_label[i]/number_docs for i in count_p_label} 
    return dict_label, count_per_label


In [5]:
## format data and get the count for classes
dict_label_class, count_per_label = function_to_format_data(data)
dict_label_doc, _ = function_to_format_data(doc)

{'vegetable': 2, 'flower': 3, 'fruit': 3}
{'D1': 1, 'D2': 1}


In [6]:
## probability per class
count_per_label

{'flower': 0.375, 'fruit': 0.375, 'vegetable': 0.25}

In [8]:
## Naive Bayes P(C/D) = (P(C)P(D/C))/P(D) but since P(D) is common to measure across all
## P(D/C) = Product over all words ((count(w,C) + 1)/ count(c) + |V|)
def prob_per_class(doc_words, dict_label_class, class_id):
    vocab = len(set([i for j in dict_label_class.values() for i in j]))
    prob_d_c = 1
    lambda_val = 0.1
    for i in doc_words:
        prob_d_c *= (dict_label_class[class_id].count(i) + lambda_val)/(len(dict_label_class[class_id]) + vocab*lambda_val)
    return prob_d_c

In [9]:
## calculate naive bayes for each class
dict_doc_class = {}
prob_c = {}
print (dict_label_doc)
for doc in dict_label_doc:
    per_doc_class = {}
    for class_id in dict_label_class:
        doc_words = dict_label_doc[doc]
        prod_d_c = prob_per_class(doc_words, dict_label_class, class_id)
        prod_d_c *= count_per_label[class_id]
        per_doc_class[class_id] = prod_d_c
    print (per_doc_class)
    dict_doc_class[doc] = max(per_doc_class, key = per_doc_class.get)
dict_doc_class

{'D1': ['rose', 'lily', 'apple', 'carrot'], 'D2': ['pea', 'pea', 'lotus', 'grape']}
{'vegetable': 3.522258934549205e-08, 'flower': 5.85199329043067e-07, 'fruit': 1.8635701033660223e-05}
{'vegetable': 1.5533161901361992e-05, 'flower': 1.0552774786022522e-07, 'fruit': 1.5401405812942332e-07}


{'D1': 'fruit', 'D2': 'vegetable'}